In [47]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor

In [48]:
import os
seed=42
np.random.seed(seed)
dir_path='C:\\Data Science\\kaggle regr competion'
file_name='preprocessedtrain.csv'
dataset=pd.read_csv(os.path.join(dir_path,file_name))

In [49]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328 entries, 0 to 1327
Columns: 219 entries, MSSubClass to PavedDrive_Y
dtypes: float64(3), int64(216)
memory usage: 2.2 MB


In [50]:
dataset.head(10)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y
0,60,65.0,8450,7,5,196.0,706,0,150,856,...,0,0,0,1,0,0,0,1,0,1
1,20,80.0,9600,6,8,0.0,978,0,284,1262,...,0,0,0,1,0,0,0,1,0,1
2,60,68.0,11250,7,5,162.0,486,0,434,920,...,0,0,0,1,0,0,0,1,0,1
3,70,60.0,9550,7,5,0.0,216,0,540,756,...,0,0,0,1,0,0,0,1,0,1
4,60,84.0,14260,8,5,350.0,655,0,490,1145,...,0,0,0,1,0,0,0,1,0,1
5,50,85.0,14115,5,5,0.0,732,0,64,796,...,0,0,0,1,0,0,0,1,0,1
6,20,75.0,10084,8,5,186.0,1369,0,317,1686,...,0,0,0,1,0,0,0,1,0,1
7,50,51.0,6120,7,5,0.0,0,0,952,952,...,1,0,0,0,0,0,0,1,0,1
8,190,50.0,7420,5,6,0.0,851,0,140,991,...,0,1,0,0,0,0,0,1,0,1
9,20,70.0,11200,5,5,0.0,906,0,134,1040,...,0,0,0,1,0,0,0,1,0,1


In [58]:
opt_predictors=['MasVnrType_Stone',
 'ExterQual_Gd',
 'Age',
 'WoodDeckSF',
 'Foundation_PConc',
 'EnclosedPorch',
 'BsmtQual_Gd',
 'BldgType_Twnhs',
 'KitchenQual_TA',
 'BsmtFinSF2',
 'Exterior1st_CemntBd',
 'GarageFinish_Unf',
 'Exterior1st_VinylSd',
 'GarageType_Attchd',
 'BsmtFullBath',
 'HalfBath',
 'FireplaceQu_no_fireplace',
 'GrLivArea',
 'TotRmsAbvGrd',
 'MSSubClass',
 'BsmtQual_TA',
 'KitchenQual_Gd',
 'Neighborhood_NridgHt',
 'FullBath',
 'BsmtFinSF1',
 'Neighborhood_NoRidge',
 'AgeGarage',
 'BsmtFinType1_Unf',
 'Foundation_CBlock',
 'GarageArea',
 'BsmtUnfSF',
 'BsmtExposure_Gd',
 'GarageFinish_RFn',
 'TotalBsmtSF',
 'Exterior1st_BrkFace',
 'HeatingQC_TA',
 'Condition1_Norm',
 'Functional_Typ',
 'MSZoning_RL',
 'GarageType_Detchd',
 'Neighborhood_Crawfor',
 'Exterior2nd_CmentBd',
 'MSZoning_RM',
 '1stFlrSF',
 '2ndFlrSF',
 'BldgType_Duplex',
 'Exterior2nd_VinylSd',
 'MasVnrArea',
 'BldgType_TwnhsE',
 'Neighborhood_StoneBr',
 'Fireplaces',
 'MasVnrType_None',
 'KitchenQual_Fa',
 'ScreenPorch',
 'MSZoning_FV',
 'AgeRemod',
 'GarageType_BuiltIn',
 'LotArea',
 'OpenPorchSF',
 'BsmtFinType1_GLQ',
 'OverallQual',
 'LotFrontage',
 'OverallCond',
 'BsmtQual_Fa',
 'ExterQual_TA',
 'GarageCars']

In [52]:
X=dataset[opt_predictors]
y=dataset['SalePrice']

In [53]:
corr_matrix = X.corr().abs()
high_corr_var=np.where(corr_matrix>0.63)
high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

In [54]:
high_corr_var

[('ExterQual_Gd', 'KitchenQual_TA'),
 ('ExterQual_Gd', 'KitchenQual_Gd'),
 ('ExterQual_Gd', 'ExterQual_TA'),
 ('Age', 'Foundation_PConc'),
 ('Age', 'BsmtQual_TA'),
 ('Age', 'AgeGarage'),
 ('Foundation_PConc', 'BsmtQual_TA'),
 ('Foundation_PConc', 'AgeGarage'),
 ('Foundation_PConc', 'Foundation_CBlock'),
 ('BsmtQual_Gd', 'BsmtQual_TA'),
 ('KitchenQual_TA', 'KitchenQual_Gd'),
 ('KitchenQual_TA', 'ExterQual_TA'),
 ('Exterior1st_CemntBd', 'Exterior2nd_CmentBd'),
 ('Exterior1st_VinylSd', 'Exterior2nd_VinylSd'),
 ('GarageType_Attchd', 'GarageType_Detchd'),
 ('BsmtFullBath', 'BsmtFinSF1'),
 ('HalfBath', '2ndFlrSF'),
 ('FireplaceQu_no_fireplace', 'Fireplaces'),
 ('GrLivArea', 'TotRmsAbvGrd'),
 ('GrLivArea', '2ndFlrSF'),
 ('GrLivArea', 'SalePrice'),
 ('BsmtFinSF1', 'BsmtFinType1_Unf'),
 ('GarageArea', 'GarageCars'),
 ('GarageArea', 'SalePrice'),
 ('TotalBsmtSF', '1stFlrSF'),
 ('MSZoning_RL', 'MSZoning_RM'),
 ('MasVnrArea', 'MasVnrType_None'),
 ('OverallQual', 'ExterQual_TA'),
 ('OverallQual', '

In [55]:
toxic_features=['KitchenQual_TA','KitchenQual_Gd','ExterQual_TA',
'Foundation_PConc','BsmtQual_TA','AgeGarage',
'Exterior2nd_CmentBd','Exterior2nd_VinylSd',
'GarageType_Detchd','2ndFlrSF','TotRmsAbvGrd',
'Fireplaces','GarageCars','TotalBsmtSF','MasVnrType_None',
'KitchenQual_TA','BsmtFullBath','BsmtFinType1_Unf','MSZoning_RL']

In [56]:
X.drop(columns=toxic_features,axis=1,inplace=True)

C:\Users\moisessalazar77\Anaconda3\envs\regression_competition\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [82]:
from sklearn.model_selection import cross_val_score

def regressor_fitness_report(regressor, X, y, num_validations):
    Rsquare = cross_val_score(regressor, 
            X, y, scoring='r2', cv=num_validations)
    print("R square: " + str(round(100*Rsquare.mean(), 2)) + "%")
    explained_var= cross_val_score(regressor, 
            X, y, scoring='explained_variance', cv=num_validations)
    print("The explained variance is: " + str(round(100*explained_var.mean(), 2)) + "%")
    MAE = cross_val_score(regressor, 
            X, y, scoring='neg_median_absolute_error', cv=num_validations)
    print("The median absolute error is: " + str(round(-1*(MAE.mean()), 2)))
    RMSE = cross_val_score(regressor, 
            X, y, scoring='neg_mean_squared_error', cv=num_validations)
    print("The root mean squared error is: " + str(round((-1*(RMSE.mean()))**0.5, 2)))
    LOGRMSE = cross_val_score(regressor, 
            X, y, scoring='neg_mean_squared_log_error', cv=num_validations)
    print("The log mean squared error is: " + str(round(-1*(LOGRMSE.mean()), 2)))

In [83]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [84]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [85]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
imp = pd.DataFrame(xgb.feature_importances_ ,columns = ['Importance'],index = X_train.columns)
imp = imp.sort_values(['Importance'], ascending = False)

print(imp)

                          Importance
GrLivArea                   0.115326
LotArea                     0.083460
Age                         0.081942
BsmtFinSF1                  0.072838
1stFlrSF                    0.068285
OverallQual                 0.065250
GarageArea                  0.054628
OverallCond                 0.053111
BsmtUnfSF                   0.050076
AgeRemod                    0.048558
WoodDeckSF                  0.037936
BsmtQual_Gd                 0.022762
OpenPorchSF                 0.022762
BsmtFinSF2                  0.016692
LotFrontage                 0.016692
MasVnrArea                  0.015175
Neighborhood_Crawfor        0.015175
Neighborhood_StoneBr        0.015175
ScreenPorch                 0.013657
MSZoning_RM                 0.012140
Functional_Typ              0.012140
Condition1_Norm             0.010622
FireplaceQu_no_fireplace    0.010622
Exterior1st_BrkFace         0.010622
MasVnrType_Stone            0.009105
BsmtFinType1_GLQ            0.007587
G

In [86]:
regressor_fitness_report(xgb, X_train, y_train,3)

R square: 88.75%
The explained variance is: 88.76%
The median absolute error is: 10998.65
The root mean squared error is: 21733.57
The log mean squared error is: 0.02


In [87]:
 #Evaluate performance of Decision Tree regressor
from sklearn.metrics import mean_squared_error, explained_variance_score
y_pred_xgb = xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred_xgb)
evs = explained_variance_score(y_test, y_pred_xgb) 
print("\n#### XGboost model performance ####")
print("Mean squared error =", round(mse, 2))
print("Explained variance score =", round(evs, 2))


#### Decision Tree performance ####
Mean squared error = 418093657.25
Explained variance score = 0.91


In [88]:
csv_file_name='hold_on_data.csv'
dataset2=pd.read_csv(os.path.join(dir_path,csv_file_name))

In [89]:
dataset.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
5,1466,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
6,1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal
7,1468,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
8,1469,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
9,1470,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal
